This Notebook was used to look at making the Scene object process rays in parallel.

Created during an Undergraduate Research Experience Programme placement at Imperial College London 2020
by Jakub Dranczewski.

To contact me, try (in no particular order)
* jbd17@ic.ac.uk (unless I left)
* jakub.dranczewski@gmail.com
* jdranczewski.github.io (there should be an email in the CV)
* some other social media platform

This code should also be available at https://github.com/jdranczewski/optical-levitation-raytracing-experiments

In [1]:
from sys import path
path.insert(0,'..')

import multiprocessing as mp
import numpy as np

In [ ]:
def f(x):
    print(x)
    return x

if __name__ == '__main__':
    with mp.Pool(5) as p:
        print(p.map(f, np.linspace(0,1,20), 2))

In [ ]:
s = slice(1,3)
a = np.arange(0,10,1)

In [ ]:
a[1:3]

In [ ]:
a[s]

In [ ]:
print(s)

# Multi Scene

In [2]:
import ptracer as pt
from importlib import reload
%matplotlib notebook
import matplotlib.pyplot as plt
%load_ext snakeviz

In [12]:
reload(pt)

<module 'ptracer' from '/home/jdranczewski/Documents/Imperial/UROP20/optical-levitation-raytracing-experiments/ptracer.py'>

In [6]:
class MultiScene:
    def __init__(self, rf, obj, n_threads=5):
        self.scenes = []
        batch = int(np.ceil(len(rf.origins) / n_threads))
        for i in range(n_threads):
            rf2 = pt.RayFactory()
            s = slice(batch*i, batch*(i+1))
            rf2.origins = rf.origins[s]
            rf2.dirs = rf.dirs[s]
            rf2.weights = rf.weights[s]
            rf2.wavelength = rf.wavelength
            self.scenes.append(pt.Scene(rf2, obj))

In [ ]:
m = pt.Sphere([1e-6,2e-6,50e-6],5e-6,n_in=1.52, reflective=False)
rf = pt.AdaptiveGaussianRF([0,0,0],[0,0,1],3.4e-6,300e-3,300,532,[1e-6,2e-6,-5.1e-6+50e-6],5e-6)
ms = MultiScene(rf, [m])

In [ ]:
ps_parts_acc = []
for scene in ms.scenes:
    scene.run(100)
    ps_parts_acc.append(scene.momentum)

In [ ]:
ps_parts_acc

In [ ]:
np.array(ps_parts_acc)[1:] - np.array(ps_parts_acc)[:-1]

In [31]:
ps_parts

[array([-2.17195583e+13, -1.92580473e+13,  7.54600766e+13]),
 array([-1.21451837e+13, -4.04899420e+13,  5.79401007e+13]),
 array([-7.36346146e+13, -1.15044873e+14,  1.69491411e+14]),
 array([-3.34125967e+13, -9.90494863e+13,  1.29104608e+14]),
 array([-9.24423757e+13, -1.52660385e+14,  2.48671285e+14])]

In [ ]:
np.sum(ps_parts, axis=0)

Normal scene gives [-8.33919559e-11 -1.66784159e-10  2.50316261e-10]

The usual result is [-1.25854323e+14 -2.51709019e+14  3.77774849e+14]

In [ ]:
m = pt.Sphere([1e-6,2e-6,50e-6],5e-6,n_in=1.52, reflective=False)
rf = pt.AdaptiveGaussianRF([0,0,0],[0,0,1],3.4e-6,300e-3,300,532,[1e-6,2e-6,-5.1e-6+50e-6],5e-6)
ms = MultiScene(rf, [m])

q = mp.Queue()
for scene in ms.scenes:
    p = mp.Process(target=scene.run, args=(100,q))
    p.start()
p.join()

ps = []
for i in range(len(ms.scenes)):
    ps.append(q.get())

In [ ]:
ps

In [ ]:
ps_parts_acc[-1]

In [ ]:
np.sum(ps, axis=0)

The usual result is [-1.25854323e+14 -2.51709019e+14  3.77774849e+14]

In [ ]:
for scene in ms.scenes:
    print(len(scene.r_origins))

# Snakeviz

In [9]:
%%snakeviz
m = pt.MeshTO([1e-6,2e-6,50e-6], "cruel_sphere.obj",5e-6,n_in=1.52, reflective=False)
rf = pt.AdaptiveGaussianRF([0,0,0],[0,0,1],3.4e-6,300e-3,300,532,[1e-6,2e-6,-5.1e-6+50e-6],5e-6)
s = pt.Scene(rf, [m])
s.run(limit=5)
print(s.momentum)

[-1.17125612e+14 -2.31801866e+14  3.40500708e+14]
 
*** Profile stats marshalled to file '/tmp/tmpyp9vvdkh'. 
Embedding SnakeViz in this document...


In [17]:
%%snakeviz
m = pt.MeshTO([1e-6,2e-6,50e-6], "cruel_sphere.obj",5e-6,n_in=1.52, reflective=False)
rf = pt.AdaptiveGaussianRF([0,0,0],[0,0,1],3.4e-6,300e-3,300,532,[1e-6,2e-6,-5.1e-6+50e-6],5e-6)
ms = MultiScene(rf, [m], 5)

q = mp.Queue()
for scene in ms.scenes:
    p = mp.Process(target=scene.run, args=(5,q))
    p.start()
p.join()

ps = np.zeros(3)
for i in range(len(ms.scenes)):
    ps += q.get()
print(ps)

[-1.17125594e+14 -2.31801449e+14  3.40501486e+14]
 
*** Profile stats marshalled to file '/tmp/tmpkb0oaml4'. 
Embedding SnakeViz in this document...


## Mesh

In [62]:
%%timeit
m = pt.MeshTO([1e-6,2e-6,50e-6], "simple_sphere.obj",5e-6,n_in=1.52, reflective=False)
rf = pt.AdaptiveGaussianRF([0,0,0],[0,0,1],3.4e-6,300e-3,1000,532,[1e-6,2e-6,-5.1e-6+50e-6],5e-6)
s = pt.Scene(rf, [m])
s.run(5)
# print(s.momentum)

205 ms ± 1.93 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [61]:
%%timeit
m = pt.MeshTO([1e-6,2e-6,50e-6], "simple_sphere.obj", 5e-6,n_in=1.52, reflective=False)
rf = pt.AdaptiveGaussianRF([0,0,0],[0,0,1],3.4e-6,300e-3,1000,532,[1e-6,2e-6,-5.1e-6+50e-6],5e-6)
s = pt.MultiScene(rf, [m], 8)
s.run(5)
# print(s.momentum)

96.9 ms ± 2.89 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Sphere

In [44]:
%%timeit
m = pt.Sphere([1e-6,2e-6,50e-6],5e-6,n_in=1.52, reflective=False)
rf = pt.AdaptiveGaussianRF([0,0,0],[0,0,1],3.4e-6,300e-3,1000,532,[1e-6,2e-6,-5.1e-6+50e-6],5e-6)
s = pt.Scene(rf, [m])
s.run(5)
# print(s.momentum)

5.22 ms ± 47.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [45]:
%%timeit
m = pt.Sphere([1e-6,2e-6,50e-6],5e-6,n_in=1.52, reflective=False)
rf = pt.AdaptiveGaussianRF([0,0,0],[0,0,1],3.4e-6,300e-3,1000,532,[1e-6,2e-6,-5.1e-6+50e-6],5e-6)
s = pt.MultiScene(rf, [m])
s.run(5)
# print(s.momentum)

34.7 ms ± 371 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


# Whyyyy

In [31]:
m = pt.Sphere([1e-6,2e-6,50e-6],5e-6,n_in=1.52, reflective=False)
rf = pt.AdaptiveGaussianRF([0,0,0],[0,0,1],3.4e-6,300e-3,300,532,[1e-6,2e-6,-5.1e-6+50e-6],5e-6)
s = pt.Scene(rf, [m])
ms = pt.MultiScene(rf, [m])

In [43]:
[sc.r_wavelength for sc in ms.scenes]

[532, 532, 532, 532, 532]

In [42]:
s.r_wavelength

532